In [159]:
import pickle
with open('../data/data.pickle', 'rb') as f:
  data = pickle.load(f)

In [173]:
texts = data['fixcapital']
texts

0            V 76 sz XAo Q 9
1                milan ek 22
2             Pan charT 1983
3              pi pink a 123
4                    bulldog
5        nere knu 9558285913
6                  bl by sek
7                 123456 a 7
8                 lumpy 1976
9                  simon m 4
10                 kur vicky
11               MlGqTAZY 05
12         PeMa KuH a 2834 @
13                 fer b man
14                    131619
15                Detail ist
16                debus trol
17              mixvibes 123
18                    161616
19                hob lly 11
20                Stano 2724
21                  19901807
22                   2925845
23              je sen jet u
24                    565455
25                UbOpriQPjq
26           MvL JB 3 e 8 zs
27                  12082389
28                tomas 2008
29               jde mena to
                ...         
38620             OwzITceVfE
38621                  27303
38622      to mask o chelsea
38623         

In [131]:
def flat(l):
    for k in l:
        if not isinstance(k, (list, tuple)):
            yield k
        else:
            yield from flat(k)
texts = list(flat(texts))

In [152]:
print(str(len(texts)) + ' texts')
range(0,1000,38000)

1 texts


range(0, 1000, 38000)

First step: Deal with password without capital words

In [199]:
from sklearn.feature_extraction.text import TfidfVectorizer

# change the max_df to get different results

tfidf_vectorizer = TfidfVectorizer(max_df=0.9999, max_features=200000,
                                   min_df=0.0001, stop_words=my_stop_words, 
                                   use_idf=True, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

print(tfidf_matrix.shape)


CPU times: user 365 ms, sys: 5.64 ms, total: 371 ms
Wall time: 370 ms
(38650, 3652)


In [200]:
terms = tfidf_vectorizer.get_feature_names()

In [201]:
terms

['00',
 '000',
 '0000',
 '001',
 '002',
 '003',
 '005',
 '007',
 '008',
 '009',
 '01',
 '0101',
 '011',
 '012',
 '0123',
 '0123456789',
 '02',
 '023',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '100',
 '1000',
 '101',
 '1010',
 '1012',
 '102',
 '105',
 '107',
 '11',
 '110',
 '1109',
 '111',
 '1111',
 '111111',
 '112',
 '1122',
 '112233',
 '113',
 '114',
 '117',
 '118',
 '1188',
 '12',
 '120',
 '1200',
 '1205',
 '1208',
 '121',
 '1212',
 '122',
 '1224',
 '123',
 '123 abc',
 '1231',
 '123123',
 '12321',
 '123321',
 '1234',
 '12344321',
 '12345',
 '123456',
 '1234567',
 '12345678',
 '123456789',
 '1235',
 '123654',
 '123654789',
 '123789',
 '124',
 '125',
 '128',
 '129',
 '13',
 '1302',
 '131',
 '132',
 '133',
 '13579',
 '138',
 '14',
 '1411',
 '142',
 '144',
 '146',
 '147',
 '147258369',
 '148',
 '1488',
 '15',
 '150',
 '1501',
 '1502',
 '151',
 '1512',
 '1515',
 '152',
 '155',
 '156',
 '157',
 '158',
 '159',
 '159357',
 '159753',
 '16',
 '1605',
 '1608',
 '1610',
 '1616',


from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [193]:
from sklearn.cluster import KMeans
# num of clusters depends on dataset 
num_clusters = 10
km = KMeans(n_clusters=num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

CPU times: user 12.8 s, sys: 11.3 ms, total: 12.8 s
Wall time: 2.56 s


Try to get a matrix

In [197]:
import pandas as pd
df_tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray())
df_tfidf_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,212,213,214,215,216,217,218,219,220,221
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
coun = 0
for i in clusters:
    if(i == 0):
        coun = coun+1
coun

974

In [19]:
print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    for ind in order_centroids[i, :6]:
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')

Top terms per cluster:

Cluster 0 words:

NameError: name 'vocab_frame' is not defined

https://github.com/llSourcell/Latent_Dirichlet_Allocation/blob/master/NewAnalysis-Normal.ipynb